In [ ]:
import random
import torch

class ImagePool:
    """
    이미지 풀 클래스:
    - 과거 생성된 이미지를 저장하고 랜덤으로 선택하거나 새 이미지를 추가.
    - GAN 훈련에서 생성자 안정성을 높이기 위해 사용.
    """
    def __init__(self, pool_size):
        """
        초기화:
        Args:
            pool_size (int): 이미지 풀의 최대 크기.
        """
        self.pool_size = pool_size
        if self.pool_size > 0:
            self.num_imgs = 0
            self.images = []

    def query(self, images):
        """
        이미지 풀에서 랜덤으로 이전 이미지를 반환하거나 새 이미지 저장.
        Args:
            images (torch.Tensor): 입력 이미지 배치 [batch_size, C, H, W].

        Returns:
            torch.Tensor: 이미지 배치 [batch_size, C, H, W].
        """
        if self.pool_size == 0:
            return images  # 풀 크기가 0이면 입력 그대로 반환.

        return_images = []
        for i in range(images.size(0)):
            image = images[i:i + 1].detach()  # 배치 차원을 유지하며 그래프에서 분리.
            if self.num_imgs < self.pool_size:
                # 풀 크기 미만: 새 이미지를 추가.
                self.num_imgs += 1
                self.images.append(image)
                return_images.append(image)
            else:
                # 풀 크기 초과: 랜덤으로 교체하거나 그대로 반환.
                if random.uniform(0, 1) > 0.5:
                    random_id = random.randint(0, self.pool_size - 1)
                    tmp = self.images[random_id].clone()  # 랜덤 선택된 이미지.
                    self.images[random_id] = image  # 풀에서 새 이미지로 교체.
                    return_images.append(tmp)
                else:
                    return_images.append(image)  # 새 이미지 그대로 반환.

        # 배치 형태로 반환.
        return torch.cat(return_images, dim=0)

import torch
import torch.nn.functional as F

def Mean_Squared_Error(tensorA, tensorB):
    """
    MSE 메트릭 계산
    :param tensorA: NCHW 형태의 텐서
    :param tensorB: NCHW 형태의 텐서
    :return: MSE 값
    """
    if tensorA.dim() != 4 or tensorB.dim() != 4:
        raise ValueError(f"Expected input tensors to be 4D (NCHW), but got shapes {tensorA.shape} and {tensorB.shape}")
    mse = F.mse_loss(tensorA, tensorB)
    return mse

def Peak_Signal_to_Noise_Rate(tensorA, tensorB, PIXEL_MAX=1.0):
    """
    PSNR 메트릭 계산
    :param tensorA: NCHW 형태의 텐서
    :param tensorB: NCHW 형태의 텐서
    :param PIXEL_MAX: 최대 픽셀 값
    :return: PSNR 값
    """
    if tensorA.dim() != 4 or tensorB.dim() != 4:
        raise ValueError(f"Expected input tensors to be 4D (NCHW), but got shapes {tensorA.shape} and {tensorB.shape}")

    mse = torch.mean((tensorA - tensorB) ** 2)
    if mse == 0:
        return float('inf')  # 두 텐서가 완전히 동일한 경우 PSNR은 무한대
    psnr = 20 * torch.log10(PIXEL_MAX / torch.sqrt(mse))
    return psnr

def Cosine_Similarity(tensorA, tensorB):
    """
    코사인 유사도 메트릭 계산
    :param tensorA: NCHW 형태의 텐서
    :param tensorB: NCHW 형태의 텐서
    :return: 코사인 유사도 값
    """
    if tensorA.dim() != 4 or tensorB.dim() != 4:
        raise ValueError(f"Expected input tensors to be 4D (NCHW), but got shapes {tensorA.shape} and {tensorB.shape}")

    tensorA_flat = tensorA.view(tensorA.size(0), -1)
    tensorB_flat = tensorB.view(tensorB.size(0), -1)

    # 코사인 유사도 계산
    cosine_sim = F.cosine_similarity(tensorA_flat, tensorB_flat, dim=1)
    return torch.mean(cosine_sim)

# 테스트 코드
if __name__ == "__main__":
    import numpy as np

    # 랜덤 테스트 데이터 생성
    a = torch.rand(2, 3, 10, 10)
    b = torch.rand(2, 3, 10, 10)

    print("MSE:", Mean_Squared_Error(a, b))
    print("PSNR:", Peak_Signal_to_Noise_Rate(a, b, PIXEL_MAX=1.0))
    print("Cosine Similarity:", Cosine_Similarity(a, b))


